Read from music dataset and write it into 

In [ ]:
!pip install pandas

In [ ]:
# make working directory if not present
!mkdir -p raw/datasets/
%cd ./raw/datasets/
# fetch datasource for 
!wget -N https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/CDs_and_Vinyl.jsonl.gz
!wget -N https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/meta_categories/meta_CDs_and_Vinyl.jsonl.gz

In [ ]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

# add filter during loading
def getDF(path, limit=-1, filter=None):
  i = 0
  df = {}
  for d in parse(path):
    if(limit >0):
      if(i>=limit):
        break
    if(filter and filter(d)):
      #print(d["title"] + "|"+str(d["average_rating"])+"|"+str(d["rating_number"]))
      continue
    else:
      df[i] = d
      i += 1

  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
def filterByRating(data, average_rating=3, rating_number=20):
    if(data['average_rating'] and  data['rating_number']):
        return data['average_rating']< average_rating or data['rating_number'] < rating_number
    else:
        return False

def filterByASIN(asin_list=[]):
    def filterInList(data):
        if(data['asin']):
            return (data['asin'] not in asin_list)
        else:
            return True
    return filterInList

def filterByParentASIN(asin_list=[]):
    def filterInList(data):
        if(data['parent_asin']):
            return (data['parent_asin'] not in asin_list)
        else:
            return True
    return filterInList
  

In [ ]:
FIELD_CATEGORY = 'categories' #for dataset older than 2023, use 'category'
FIELD_JOIN_KEY = 'parent_asin' #for dataset older than 2023, use 'asin'

RATINGS_FILE_NAME = "ratings.dat"
CONTENT_FILE_NAME = "music.dat"

SEPERATOR = ";"

# making working directory for music
!mkdir -p music
%pwd

In [ ]:
import re
import collections

unbroken_category = ["International Music","Indie & Alternative","British Isles"]
filtered_token = ["Music"]
synonym_mapping = {
    "Classical": ["Classic"],
    "International Music": ["International"],
}
def tokenizeCategory(row):
    result = []
    for ele in row:
        if ele in unbroken_category:
            continue
        tokens = re.split(r" & | ",ele)
        # skip the token if it is already in row
        token = [item for item in tokens if item in row]
        
        #filter out the field tokens that is the same as the category
        if(len(tokens) == 1):
            continue
        else:
            #eliminate redundent or misleading tokens 
            tokens = [item.strip() for item in tokens if item not in filtered_token]
            #eliminate redundent or misleading tokens 
            tokens = [item for item in list(dict.fromkeys(tokens))]
            #may to synonym
            for key, value in synonym_mapping.items():
                tokens = [key if item in value else item for item in tokens]
            
            result.extend(tokens)
    return result

# Step 1: Read the json file into a DataFrame
input_file = 'meta_CDs_and_Vinyl.jsonl.gz'
df = getDF(input_file, 1000, filterByRating) #enabled for 2023 data
# set music_id according to the since the model requires sequence for music_id
# df["music_id"] = df["parent_asin"].apply(hashToInt)
df.sort_values(by="rating_number")
df["music_id"] = df.reindex().index +1
# Step 2: 
# Filter out default genres
genres_to_be_filtered = ["CDs & Vinyl","Today's Deals in Music","CDs $7 - $10","CDs Under $7","Tapes Under $6"]
df[FIELD_CATEGORY] = df[FIELD_CATEGORY].apply(lambda row: [ele for ele in row if ele not in genres_to_be_filtered])
# Filter rows where 'category' is non-empty or an empty array
filtered_content = df[df[FIELD_CATEGORY].apply(lambda row: isinstance(row, list) and len(row) > 0)]
# Show the filtered DataFrame
print(filtered_content[FIELD_CATEGORY].count())

# tokenize category
FIELD_TOKENS = '_tok_cat'
filtered_content[FIELD_TOKENS]=filtered_content[FIELD_CATEGORY].apply(tokenizeCategory)
print(filtered_content[FIELD_TOKENS][0:5])

# tokenize the list in 'category' and '_tok_cat' and concat them with '|' as a sepa
filtered_content['genres'] = filtered_content[FIELD_CATEGORY].apply(lambda row: '|'.join(map(str, row))) 
filtered_content['genres'] = filtered_content['genres'] + '|' + filtered_content[FIELD_TOKENS].apply(lambda row: '|'.join(map(str, row)) ) 
# remove the last '|' if it is the last character
filtered_content['genres'] = filtered_content['genres'].apply(lambda row: row[:-1] if row[-1] == '|' else row)

# Step 3: Pick necessary columns and save to csv file
selected_content = ["music_id", "title", "genres"]#enabled for 2023 data
df_content = filtered_content[selected_content]
content_file = 'music/{file_name}'.format(file_name = CONTENT_FILE_NAME)
df_content.to_csv(content_file, sep=SEPERATOR, index=False, header=False)

In [ ]:
# # Step 1: Read the json file into a DataFrame
input_file = 'CDs_and_Vinyl.jsonl.gz'

# step 2: Preprocess the data 
asin_list=filtered_content[FIELD_JOIN_KEY].to_list()#enabled for 2023 data
rating_sum = filtered_content["rating_number"].sum()
print(f"Fetch comments for  {len(asin_list)} products and supposed ratings to be recieved= {rating_sum}")
df = getDF(input_file,rating_sum, filterByParentASIN(asin_list))#enabled for 2023 data
# TODO: sum the ratings by parent_asin to see the different in rating_number and actually ratings counted

df = df.set_index(FIELD_JOIN_KEY).join(filtered_content.set_index(FIELD_JOIN_KEY), rsuffix="meta_")

# Step 3: Pick necessary columns and save to csv 
selected_rating = ["user_id", "music_id", "rating","timestamp"] #enabled for 2023 data
df_rating = df[selected_rating]
rating_file = 'music/{file_name}'.format(file_name = RATINGS_FILE_NAME)
df_rating.to_csv(rating_file, sep=SEPERATOR, index=False, header=False)

In [ ]:
import os
RATINGS_DATA_COLUMNS = ["UserID", "MovieID", "Rating", "Timestamp"]
MOVIES_DATA_COLUMNS = ["MovieID", "Title", "Genres"]

print("First few lines for ratings and movies:")

# read the data 
ratings_df = pd.read_csv(
    os.path.join(os.curdir, "music",RATINGS_FILE_NAME),
    sep=SEPERATOR,
    names=RATINGS_DATA_COLUMNS,
    encoding="unicode_escape")

print(ratings_df.head())

movies_df = pd.read_csv(
    os.path.join(os.curdir, "music", CONTENT_FILE_NAME),
    sep=SEPERATOR,
    names=MOVIES_DATA_COLUMNS,
    encoding="unicode_escape")  # May contain unicode. Need to escape.

print(movies_df.head())